In [20]:
#import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import os

proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset1.csv')
df = pd.read_csv(train_path)

labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

oversample = SMOTENC(categorical_features=[1, 2,17,18,19])
x_train, y_train = oversample.fit_resample(x_train, y_train)

sc = RobustScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=20, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,
      decay_steps=(x_train.shape[0]/32)*50,
      decay_rate=1,
      staircase=False)

def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)

nn_model.compile(loss = "binary_crossentropy", 
                  optimizer = get_optimizer(), 
                  metrics=['accuracy'])

callback =tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
history = nn_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])

y_pred = nn_model.predict(x_test)

print ('accuracy : {0:.2f}'.format(accuracy_score(y_test,y_pred.round())))
print ('precision  : {0:.2f}'.format(precision_score(y_test,y_pred.round())))
print ('recall  : {0:.2f}'.format(recall_score(y_test,y_pred.round())))
print ('f1 score  : {0:.2f}'.format(f1_score(y_test,y_pred.round())))
print ('confusion matrix : \n {0}'.format(confusion_matrix(y_test,y_pred.round())))

accuracy : 0.92
precision  : 0.75
recall  : 0.71
f1 score  : 0.73
confusion matrix : 
 [[1041   45]
 [  55  134]]


In [21]:
ii = 1

nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=20, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
    
cv = StratifiedKFold(n_splits=10)

for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train.to_numpy())
	shap_values = explainer.shap_values(Xi_test.to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['feature_importance_vals']).to_string(index=False))
	ii += 1

Iteration 1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 feature_importance_vals
                0.008735
                0.000596
                0.000133
                0.029818
                0.063464
                0.013242
                0.002136
                0.014243
                0.018552
                0.002674
                0.012731
                0.021781
                0.000412
                0.002376
             

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 feature_importance_vals
                0.006146
                0.003994
                0.000813
                0.018930
                0.056331
                0.012673
                0.006254
                0.011211
                0.011505
                0.005654
                0.008303
                0.012398
                0.000252
                0.003481
                0.007190
